# <span style="color:green"> Import libraries<span style="color:green">

In [2]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

2024-12-20 09:53:20.860974: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 09:53:20.861052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 09:53:20.862438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:53:20.871958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-20 09:53:21.754965: W tensorflow/compiler/tf2

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [4]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/ldlGWAS/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

     FID  IID  PAT  MAT  SEX  PHENOTYPE  rs2980319_T  rs6685064_C  \
0  10004    1    0    0    2         75            2            1   
1  10005    1    0    0    1         69            2            2   
2  10007    1    0    0    1        108            2            2   
3  10008    1    0    0    1         94            2            2   
4  10009    1    0    0    1         92            2            2   

   rs2281173_G  rs10907187_G  ...  rs17001322_G  rs4141437_G  rs1033665_A  \
0            1             2  ...             2            2            2   
1            2             2  ...             2            2            2   
2            2             0  ...             2            2            2   
3            2             2  ...             2            2            2   
4            2             2  ...             2            2            2   

   rs5771133_T  rs3810974_C  rs138228_A  rs9617081_A  rs12484907_G  \
0            2            2           0            1

In [5]:
new_col_names= list(df.columns[:6])
new_col_names.extend([col_name[:-2] for col_name in df.columns[6:]])
df.columns=new_col_names
df.columns

Index(['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', 'rs2980319',
       'rs6685064', 'rs2281173', 'rs10907187',
       ...
       'rs17001322', 'rs4141437', 'rs1033665', 'rs5771133', 'rs3810974',
       'rs138228', 'rs9617081', 'rs12484907', 'rs17248301', 'rs8139476'],
      dtype='object', length=69908)

In [6]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (1282, 69902)
Shape of y (1282,)


In [7]:
#Train test data split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=2000)
print("The dimension of X_train is {}".format(X_train.shape))
print("The dimension of X_test is {}".format(X_test.shape))

The dimension of X_train is (897, 69902)
The dimension of X_test is (385, 69902)


## <span style="color:green"> LASSO <span style="color:green">

In [8]:
# Create and fit the LASSOCV model select 5000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.00045], cv=5, random_state=2000)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.377e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.548e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.00045
5003 selected SNPs
print shape of selected snps by lasso (1282, 5003)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.499e+02, tolerance: 1.605e+02
  model = cd_fast.enet_coordinate_descent(


### Linear regression

In [9]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(l1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
#print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
#print(top_f_indices)
top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
#print(top_f_features)
lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
#lasso_LR_sigSNP
lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
#lasso_LR_sig_col

Y.shape (1282,)
x_train=(897, 5003), x_test=(385, 5003), Y_train= (897,), Y_test= (385,)
r2 is  0.6357759780110592
mean_absolute_error 16.056727661782173
mean_squared_error  391.95561760695495
explained_variance_score 0.6366066443871343
Selected F-values:
Indices of selected features with top F-values:
Top selected features:


### Random Forest

In [10]:
# Create a Random Forest Regressor
 #reate a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

# Create RFECV object
rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=70)

# Fit RFECV on the training data
rfecv.fit(x_train, Y_train)

# Manually set the number of features you want to keep
desired_num_features =100
# Manually set the number of features you want to keep


# Get the selected features based on the desired number
selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# Print the indices of the selected features
print("Selected features indices:", selected_features)

# Transform the training and testing data to keep only the selected features
X_train_selected = rfecv.transform(x_train)
X_test_selected = rfecv.transform(x_test)


# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor_selected.fit(X_train_selected, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor_selected.predict(X_test_selected)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
lasso_RF_sigSNP=l1.iloc[:,selected_features]
lasso_RF_sig_col= list(lasso_RF_sigSNP.columns)
lasso_RF_sig_col

Selected features indices: [  13   14   19   43   64   70   90   91   98  122  140  169  195  199
  207  224  238  245  262  263  278  279  297  305  344  352  365  438
  451  453  469  490  519  532  553  560  595  623  635  640  646  652
  681  686  692  749  791  794  805  864  867  870  872  888  896  902
  913  928  931  940  951  975  985 1025 1028 1029 1032 1035 1039 1053
 1060 1095 1120 1127 1136 1139 1155 1169 1175 1188 1191 1195 1202 1215
 1218 1239 1241 1258 1273 1274 1281 1290 1293 1307 1319 1324 1370 1375
 1379 1389]
r2 is  0.020866559486800607
mean_absolute_error 26.32997402597403
mean_squared_error  1053.6835277922078
explained_variance_score 0.021994776157934992


['rs12085743',
 'rs11587314',
 'rs12142584',
 'rs12078796',
 'rs1193219',
 'rs161807',
 'rs12125492',
 'rs4846216',
 'rs2379152',
 'rs12758763',
 'rs2235918',
 'rs2743208',
 'rs230175',
 'rs1043424',
 'rs3738091',
 'rs1555024',
 'rs4649033',
 'rs4659362',
 'rs9426278',
 'rs9286959',
 'rs592232',
 'rs728340',
 'rs7521293',
 'rs10798977',
 'rs1110113',
 'rs2484754',
 'rs12563174',
 'rs17108782',
 'rs9332417',
 'rs744748',
 'rs4278395',
 'rs1202824',
 'rs6676078',
 'rs2782549',
 'rs17318954',
 'rs11208264',
 'rs2863200',
 'rs535112',
 'rs2422137',
 'rs1405058',
 'rs7512474',
 'rs211764',
 'rs12125903',
 'rs473027',
 'rs11162662',
 'rs569399',
 'rs284152',
 'rs10874755',
 'rs1687949',
 'rs12023867',
 'rs10493973',
 'rs6658728',
 'rs2229783',
 'rs11581564',
 'rs2494067',
 'rs10494100',
 'rs3768480',
 'rs2786152',
 'rs1264897',
 'rs416269',
 'rs4838986',
 'rs7529979',
 'rs1146316',
 'rs4079531',
 'rs2335407',
 'rs759330',
 'rs11264640',
 'rs6694377',
 'rs6699950',
 'rs12726858',
 'rs7529215'

### Support Vector Machine

In [11]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(imp_score)[::-1]
#print(f"Selected feature indices are {feature_indices}")

# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")
#lasso_SVR_sigSNP= l1.iloc[:,top_100]
#lasso_SVR_sig_col= list(lasso_SVR_sigSNP.columns)
#lasso_SVR_sig_col


r2 is  0.657599345539942
mean_absolute_error 15.4160111992304
mean_squared_error  368.4706441246004
explained_variance_score 0.6590080478335235


### XGBoost

In [12]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.045758213198292985
mean_absolute_error 26.94532488538073
mean_squared_error  1125.3810335830306
explained_variance_score -0.04165763446728543


## <span style="color:green"> Ridge<span style="color:green">

In [13]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [67847  5994 16614 ... 10679 20908 16917]
5000 selected SNPs
Maximum value of coefficient 0.17897713157981693
print shape of selected snps by Ridge (1282, 5000)


### Linear Regression

In [14]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [15]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(R1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
#Ridge_LR_sigSNP
#Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
#Ridge_LR_sig_col

r2 is  0.7941121665413645
mean_absolute_error 11.576119452686955
mean_squared_error  221.56389488084804
explained_variance_score 0.7941732648183302
Selected p-values:


### Random Forest

In [16]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=10,random_state=2000)

# Create RFECV object
rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=4)

# Fit RFECV on the training data
rfecv.fit(x_train, Y_train)

# Manually set the number of features you want to keep
desired_num_features =100
# Manually set the number of features you want to keep


# Get the selected features based on the desired number
selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# Print the indices of the selected features
print("Selected features indices:", selected_features)

# Transform the training and testing data to keep only the selected features
X_train_selected = rfecv.transform(x_train)[:, selected_features]
X_test_selected = rfecv.transform(x_test)[:, selected_features]


# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor_selected.fit(X_train_selected, Y_train)

# Make predictions on the test set using the model with selected features
y_pred = rf_regressor_selected.predict(X_test_selected)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
#Ridge_RF_sigSNP=R1.iloc[:,selected_features]
#Ridge_RF_sig_col= list(Ridge_RF_sigSNP.columns)
#Ridge_RF_sig_col

Selected features indices: [  0   1   2   3   5   6   7   9  11  13  14  15  16  17  18  19  20  21
  22  25  27  28  29  30  31  32  34  35  36  37  38  40  42  43  44  46
  47  48  51  52  54  58  59  61  63  64  65  66  68  71  74  75  77  78
  79  81  82  83  84  85  86  93  94  95 100 105 107 110 111 112 113 115
 116 118 120 122 123 124 125 126 128 130 133 134 135 137 138 140 141 142
 143 144 147 149 151 152 153 154 155 156]
r2 is  0.04247914460022373
mean_absolute_error 25.972415584415582
mean_squared_error  1030.4253854545454
explained_variance_score 0.05089197875277074


### Support Vector Machine

In [17]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(np.abs(imp_score))[::-1]
#print(f"Selected feature indices are {feature_indices}")

# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")

#Ridge_SVR_sigSNP= R1.iloc[:,top_100]
#Ridge_SVR_sig_col= list(Ridge_SVR_sigSNP.columns)
#Ridge_SVR_sig_col

r2 is  0.7943502516488743
mean_absolute_error 11.597831965819362
mean_squared_error  221.3076822487228
explained_variance_score 0.7943648189631427


### XGBoost

In [18]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.37991823914097145
mean_absolute_error 28.730934380865715
mean_squared_error  1484.9836172695495
explained_variance_score -0.3699044458112726


## <span style="color:green"> Elastic net<span style="color:green">

In [19]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.0033], cv=5, random_state= 2000)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.977e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.556e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.0033
Optimal l1_ratio:  0.5
5037 selected SNPs
print shape of selected snps by Elastic net (1282, 5037)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.031e+02, tolerance: 1.605e+02
  model = cd_fast.enet_coordinate_descent(


In [20]:
y=list(E1.columns)[:5038]
#E1.to_("/home/Elasticnet_SVR_5000_csv", index=0)
#y

### Linear Regression

In [21]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5037), x_test=(385, 5037), Y_train= (897,), Y_test= (385,)


In [22]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(E1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
#Elasticnet_LR_sigSNP
#Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
#Elasticnet_LR_sig_col

r2 is  0.864783321688312
mean_absolute_error 9.961057916168222
mean_squared_error  145.51191974929068
explained_variance_score 0.8648870835124843


## r2_score_manual

In [25]:
def r2_score_manual(y_true, y_pred):
    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the mean of the actual values
    y_mean = np.mean(y_true)
    
    # Residual sum of squares
    ss_res = np.sum((y_true - y_pred) ** 2)
    
    # Total sum of squares
    ss_tot = np.sum((y_true - y_mean) ** 2)
    
    # R² score
    r2 = 1 - (ss_res / ss_tot)
    return r2


r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)


r2 is  0.864783321688312


### Random Forest

In [26]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

# Create RFECV object
rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error', n_jobs=70)

# Fit RFECV on the training data
rfecv.fit(x_train, Y_train)

# Manually set the number of features you want to keep
desired_num_features =100
# Manually set the number of features you want to keep


# Get the selected features based on the desired number
selected_features = rfecv.get_support(indices=True)[:desired_num_features]
# Print the indices of the selected features
#print("Selected features indices:", selected_features)

# Transform the training and testing data to keep only the selected features
X_train_selected = rfecv.transform(x_train)[:, selected_features]
X_test_selected = rfecv.transform(x_test)[:, selected_features]


# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor_selected.fit(X_train_selected, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor_selected.predict(X_test_selected)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
#Elasticnet_RF_sigSNP=E1.iloc[:,selected_features]
#Elasticnet_RF_sig_col= list(Elasticnet_RF_sigSNP.columns)
#Elasticnet_RF_sig_col

r2 is  0.04446024632645307
r2 is  0.04446024632645307
mean_absolute_error 26.018259740259737
mean_squared_error  1028.2934449350648
explained_variance_score 0.04871531452940969


In [ ]:
#from sklearn.inspection import permutation_importance

#svr = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, n_jobs=70,random_state=2000)
#svr.fit(x_train, Y_train)
#y_pred= svr.predict(x_test)
#score= r2_score(Y_test, y_pred)
#print("r2 is ", score)
#h= mean_absolute_error(Y_test, y_pred)
#print("mean_absolute_error", h)
#print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
#print("explained_variance_score", explained_variance_score(Y_test, y_pred))


#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(np.abs(imp_score))[::-1]
#print(f"Selected feature indices are {feature_indices}")
# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")
#Elasticnet_SVR_sigSNP= E1.iloc[:,top_100]
#Elasticnet_SVR_sig_col= list(Elasticnet_SVR_sigSNP.columns)
#Elasticnet_SVR_sig_col

### Support Vector Regression

In [27]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(np.abs(imp_score))[::-1]
#print(f"Selected feature indices are {feature_indices}")
# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")
#Elasticnet_SVR_sigSNP= E1.iloc[:,top_100]
#Elasticnet_SVR_sig_col= list(Elasticnet_SVR_sigSNP.columns)
#Elasticnet_SVR_sig_col

r2 is  0.8981861028949196
r2 is  0.8981861028949196
mean_absolute_error 8.666417701049134
mean_squared_error  109.56588942945788
explained_variance_score 0.8982207596651699


In [ ]:
#Elasticnet_SVR_sigSNP
#l= df.iloc[:,:6]
#l
#u= pd.concat([l, Elasticnet_SVR_sigSNP], axis=1)
#l= 
# o= np.save("/home/Vaishnavi/ssss", u)
# i= pd.read_("/home/Vaishnavi/ssss")
# i

In [ ]:
#u.to_csv("/home/Elasticnet_SVR_100_csv", index=0)

In [ ]:
#o= pd.read_csv("/home/Elasticnet_SVR_100_csv")
#o

In [ ]:
#feature_indices_abs = np.argsort(np.abs(imp_score))[::-1]
#len(feature_indices_abs)

In [ ]:
#feature_indices_pos = np.argsort(imp_score)[::-1]
#feature_indices_pos

In [ ]:
#p=np.intersect1d(feature_indices_abs, feature_indices_pos)
#len(p)

In [ ]:
#num_selected_features = 150
#top_100_abs=feature_indices_abs[:num_selected_features]
#print(f"Top 100 feature indices are {feature_indices_abs}")
#top_100_pos=feature_indices_pos[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices_pos}")
#imp_score_100_abs=imp_score[top_100_abs]
#print("abs",imp_score_100_abs)
#imp_score_100_pos=imp_score[top_100_pos]
#print("pos",imp_score_100_pos)
# print(f"Top 100 fearures permutation scores are {imp_score_100}")
#p=np.intersect1d(imp_score_100_abs, imp_score_100_pos)
#len(p)
#Elasticnet_SVR_sigSNP_abs= E1.iloc[:,top_100_abs]
#print("abs", Elasticnet_SVR_sigSNP_abs)
#Elasticnet_SVR_sigSNP_pos= E1.iloc[:,top_100_pos]
#print("pos", Elasticnet_SVR_sigSNP_pos)
#p=np.intersect1d(Elasticnet_SVR_sigSNP_abs, Elasticnet_SVR_sigSNP_pos)
#p

In [ ]:
#Elasticnet_SVR_sigSNP_abs= E1.iloc[:,top_100_abs]
#Elasticnet_SVR_sig_col_abs= list(Elasticnet_SVR_sigSNP_abs.columns)
#Elasticnet_SVR_sig_col_abs

In [ ]:
#Elasticnet_SVR_sigSNP_pos= E1.iloc[:,top_100_pos]
#Elasticnet_SVR_sig_col_pos= list(Elasticnet_SVR_sigSNP_pos.columns)
#Elasticnet_SVR_sig_col_pos

In [ ]:
#p=np.unique(Elasticnet_SVR_sig_col_pos,Elasticnet_SVR_sig_col_abs)
#p

In [ ]:
#plt.hist(imp_score_100_pos)

In [ ]:
#plt.hist(imp_score_100)

In [ ]:
#plt.hist(imp_score)

### XGBoost

In [28]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.1486632285535232
r2 is  -0.1486632285535232
mean_absolute_error 27.825146048409596
mean_squared_error  1236.121117743754
explained_variance_score -0.14587096507071218


## <span style="color:green"> Mutual information <span style="color:green">

In [31]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (1282, 69902)
Shape of y (1282,)


In [32]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 69902
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs12613647    8.081041e-02
rs6986075     7.524483e-02
rs506973      7.250038e-02
rs2048756     7.244632e-02
rs7085631     7.020267e-02
                  ...     
rs356053      1.023246e-06
rs12486193    6.066328e-07
rs6443440     6.011830e-07
rs4077566     3.887198e-07
rs12117033    1.353719e-07
Length: 34154, dtype: float64>
Length of mutal info non zero 34154
print shape of selected snps by Mutual_info (1282, 5000)


### Linear Regression

In [33]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [34]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(M1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
#MI_LR_sigSNP
#MI_LR_sig_col= list(MI_LR_sigSNP.columns)
#MI_LR_sig_col

r2 is  -0.3636281264776131
r2 is  -0.3636281264776131
mean_absolute_error 31.257174085961733
mean_squared_error  1467.4531942760675
explained_variance_score -0.3617295943021641


### Random Forest

In [35]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

# Create RFECV object
rfecv = RFECV(estimator=rf_regressor, step=30, cv=5, scoring='neg_mean_squared_error')

# Fit RFECV on the training data
rfecv.fit(x_train, Y_train)

# Manually set the number of features you want to keep
desired_num_features =100
# Manually set the number of features you want to keep


# Get the selected features based on the desired number
selected_features = rfecv.support_
# Print the indices of the selected features
print("Selected features indices:", selected_features)

# Transform the training and testing data to keep only the selected features
X_train_selected = rfecv.transform(x_train)
X_test_selected = rfecv.transform(x_test)


# Get the optimal number of features
optimal_num_features = rfecv.n_features_

# Print the optimal number of features
#print(f"Optimal number of features: {optimal_num_features}")


# Train a Random Forest Regressor on the selected features
rf_regressor_selected = RandomForestRegressor(n_estimators=100, random_state=2000)
rf_regressor_selected.fit(X_train_selected, Y_train)

# Make predictions on the test set using the model with selected features
y_pred= rf_regressor_selected.predict(X_test_selected)

score= r2_score(Y_test,y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
#MI_RF_sigSNP=M1.iloc[:,selected_features]
#MI_RF_sig_col= list(MI_RF_sigSNP.columns)
#MI_RF_sig_col

Selected features indices: [False False False ... False False False]
r2 is  0.013833386852806329
r2 is  0.013833386852806329
mean_absolute_error 26.451719178162453
mean_squared_error  1061.2521980529993
explained_variance_score 0.015694003645515986


### Support Vector Regressor

In [36]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
mae= print(f"MAE is {mean_absolute_error(Y_test, y_pred)}")


#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(np.abs(imp_score))[::-1]
#print(f"Selected feature indices are {feature_indices}")

# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")
score= r2_score(Y_test,y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
#MI_SVR_sigSNP= M1.iloc[:,top_100]
#MI_SVR_sig_col= list(MI_SVR_sigSNP.columns)
#MI_SVR_sig_col

MAE is 32.07399894842477
r2 is  -0.4511933589703607
r2 is  -0.4511933589703607
mean_absolute_error 32.07399894842477
mean_squared_error  1561.6855422556678
explained_variance_score -0.45035301832945995


# XGBoost for mutual information

In [37]:
## XGBoost for mutual information
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.20577805816084016
r2 is  -0.20577805816084016
mean_absolute_error 28.32920542134867
mean_squared_error  1297.584604394098
explained_variance_score -0.20260762111723296


# Gradient Boosting for Elastic-net

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(loss='squared_error',
    learning_rate=0.01,
    n_estimators=250,
    subsample=1.0,
    criterion='squared_error',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=15,
    min_impurity_decrease=0.0,
    init=None,
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.2,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0,random_state=2000)
reg.fit(x_train, Y_train)
y_pred=reg.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


# XGBoost for elastic-net

In [ ]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# XGBoost for lasso

In [ ]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# XGBoost for ridge

In [ ]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# XGBoost for mutual information

In [ ]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

# XGBoost for all features

In [ ]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=1000,  max_depth=15, eval_metric='rmsle', random_state=2000,
                          n_jobs=10)
regressor.fit(X_train, y_train)
y_pred= regressor.predict(X_test)
score= r2_score(Y_test,y_pred)
print("r2 score",score)
h= mean_absolute_error(Y_test,y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_test, y_pred))
print("explained_variance_score", explained_variance_score(y_test, y_pred))

## <span style="color:green"> SNP Enrichment <span style="color:green">

In [ ]:
db_snp = pybiomart.Dataset('hsapiens_snp', host='http://www.ensembl.org')
db_snp.list_attributes().head(5)
#db_snp.list_filters()
snp_results=[]
for i in Elasticnet_SVR_sig_col:
    snp_ens = db_snp.query(attributes=['refsnp_id','refsnp_source_description','chr_name','chrom_start', 'phenotype_name',
                                     'consequence_type_tv', 'minor_allele', 'clinical_significance',
                                   'associated_gene',  'ensembl_gene_stable_id'],filters={'snp_filter':[i]})
    snp_results.append(snp_ens)
   
combined_df = pd.concat(snp_results, ignore_index=True)
output_csv_path = '/home/Vaishnavi/E1_SVR_Ensembl.csv'  
combined_df.to_csv(output_csv_path, index=False)
print(f"Saved combined data to {output_csv_path}")


In [ ]:
snp_results

In [ ]:
dataset = pybiomart.Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
l=[]
p=combined_df.iloc[:,9]
for i in p:
    
    o=dataset.query(attributes=['ensembl_gene_id', 'entrezgene_id','chromosome_name','description','hgnc_symbol','phenotype_description'],
             filters={'link_ensembl_gene_id': [i]})
    l.append(o)
    
gene_info= pd.concat(l)
print(gene_info)
output_csv_path = '/home/Vaishnavi/E1_SVR_phenoinfo.csv'  
gene_info.to_csv(output_csv_path, index=False)
print(f"Saved combined data to {output_csv_path}")

In [ ]:
#Import packages
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/ldlGWAS/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

In [ ]:
new_col_names= list(df.columns[:6])
new_col_names.extend([col_name[:-2] for col_name in df.columns[6:]])
df.columns=new_col_names
df.columns

In [ ]:
X= df.iloc[:,6:]
y= df.PHENOTYPE
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=2000)
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=2000)


In [ ]:
# Define the model
model = Sequential()
model.add(Dense(30, input_dim=69902, activation='relu'))
model.add(Dense(10, activation='linear'))



In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam',
              metrics=['mean_absolute_error'])



In [ ]:
history = model.fit(X_train, y_train, epochs=100,
                    batch_size=30, validation_data=(X_val, y_val))

